In [1]:
from astropy.io.fits import getdata, getheader
from nustar_lunar_pointing.tracking import get_epoch_tle, convert_nustar_time, get_moon_j2000

### Load in the attorb data and figure out which TLE row you're going to use.

In [2]:
tlefile = '../data/NusTAR.tle'
att = getdata('../data/nu60160109002A.attorb')
hdr = getheader('../data/nu60160109002A.attorb', 1)
mjdref = hdr['MJDREFI']

checktime = convert_nustar_time(att[0]['TIME'])
mindt, line1, line2 = get_epoch_tle(checktime, tlefile)
print("Time between TLE and time you want to check:", mindt)

Time between TLE and time you want to check: 0


### Loop over the entries in the attorb file and see if the Moon moves.

Good news, there's less than 3" shift now between using the positions in the attorb file and when you use the TLE to determine the position.

In [21]:
from astropy.coordinates import SkyCoord
doff = []
for ind, t0 in enumerate(att['Time']):
    t1 = convert_nustar_time(t0)

    
    ra_moon1, dec_moon1 = get_moon_j2000(t1, line1, line2)
    moon_sky1 = SkyCoord(ra_moon1, dec_moon1)

#    position = att[ind]['Position']
#    ra_moon2, dec_moon2 = get_moon_j2000(t1, line1, line2, position=position)
#    moon_sky2 = SkyCoord(ra_moon2, dec_moon2)
    
#    doff.extend([moon_sky1.separation(moon_sky2).arcsec])
    
    if ind > 100:
        break
    break
print(ra_moon1, dec_moon1)
#print(ra_moon2, dec_moon2)
print(position)

317.22829907610003 deg -13.462168800770106 deg
[-1187.07495117 -6859.23193359  -463.08862305]


### Below was just to figure out that there's a 5 second difference between time timestamps in the attorb file and the times that are actually used to report the satellite position.

In [22]:
from nustar_lunar_pointing.tracking import get_nustar_location
from numpy import sqrt

roff = []
for ind, t0 in enumerate(att['Time']):
    t1 = convert_nustar_time(t0)

    tle_position = get_nustar_location(t1, line1, line2)
    position = att[ind]['Position']
        
    dx = tle_position - position
    dr = 0.

    for val in dx:
        dr += val**2.
        
    roff.extend([sqrt(dr)])
    
    
    if ind > 10:
        break
    break

In [24]:
from nustar_lunar_pointing.tracking import eci2el
from astropy.time import Time
import astropy.units as u


position = att['Position'][0]
#print(position)
t = Time(t1)
loc = eci2el(*tle_position*u.km,t)

print(loc.to_geodetic())
print(att['SAT_LON'][0], att['SAT_LAT'][0], att['SAT_ALT'][0]*u.km - (1*u.R_earth).to(u.km))



(<Longitude 10.078673981019103 deg>, <Latitude -3.8698797306182167 deg>, <Quantity 598.5603127919608 km>)
9.82788 -3.80595 598.4435898437496 km
